In [7]:
import time
import math
import numpy as np
import torch
import torch.nn as nn
from model import WD_LSTM
from data import Corpus
from randomize_bptt import get_bptt_sequence_lengths
from helpers import Config, repackage_hidden, batchify, get_batch

In [8]:
SEED = 42
DATA = '/floyd/home/wt2/'  # '/floyd/home/ptb/'
CUDA = True
LOG_INTERVAL = 200
LR_ANNEALING_RATE = 0.5
CONFIG_NAME = 'language_model_wt2'  # 'language_model_base'
device = torch.device("cuda" if CUDA else "cpu")
args = Config(CONFIG_NAME)

In [9]:
torch.manual_seed(SEED)
np.random.seed(SEED)

In [10]:
corpus = Corpus(DATA)

In [11]:
ntokens = len(corpus.dictionary)
train_data = batchify(corpus.train, args.batch_size, device)
valid_data = batchify(corpus.valid, args.eval_batch_size, device)
test_data = batchify(corpus.test, args.test_batch_size, device)

In [12]:
model = WD_LSTM(
    ntoken=ntokens, 
    ninp=args.emsize,
    nhid=args.nhid, 
    nlayers=args.nlayers, 
    dropout=args.dropout,
    dropout_h=args.dropout_h,
    dropout_i=args.dropout_i,
    dropout_e=args.dropout_e,
    weight_drop=args.weight_drop, 
    weight_tying=args.weight_tying
).to(device)
model

WD_LSTM(
  (variational_dropout): VariationalDropout()
  (encoder): Embedding(33278, 400)
  (rnns): ModuleList(
    (0): WeightDrop(
      (module): LSTM(400, 1150)
    )
    (1): WeightDrop(
      (module): LSTM(1150, 1150)
    )
    (2): WeightDrop(
      (module): LSTM(1150, 400)
    )
  )
  (decoder): Linear(in_features=400, out_features=33278, bias=True)
)

In [13]:
lr = args.lr
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=args.weight_decay)
criterion = nn.CrossEntropyLoss()

In [14]:
def evaluate(data_source, batch_size):
    model.eval()  # disable dropout
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, args.bptt_seq_len):
            data, targets = get_batch(data_source, i, args.bptt_seq_len)
            output, hidden, _, _ = model(data, hidden)
            hidden = repackage_hidden(hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [15]:
def ar_tar_regularization_loss(outputs, dropped_outputs):
    reg_loss = 0
    for i in range(len(dropped_outputs[-1])):
        reg_loss += args.alpha * dropped_outputs[-1][i].pow(2).mean()
        if i >= 1:
            reg_loss += args.beta * (outputs[-1][i] - outputs[-1][i - 1]).pow(2).mean()
    return reg_loss

In [16]:
def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(args.batch_size)
    for batch, (i, seq_len, lr_scale) in enumerate(get_bptt_sequence_lengths(
        train_data.size(0), 
        args.bptt_seq_len, 
        args.bptt_random_scaling, 
        args.bptt_p, 
        args.bptt_s, 
        args.bptt_min_len
    )):
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr * lr_scale
        data, targets = get_batch(train_data, i, seq_len)
        hidden = repackage_hidden(hidden)
        optimizer.zero_grad()
        output, hidden, outputs, dropped_outputs = model(data, hidden)
        output_flat = output.view(-1, ntokens)
        unregularized_loss = criterion(output_flat, targets)
        regularized_loss = unregularized_loss + ar_tar_regularization_loss(outputs, dropped_outputs)
        regularized_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()
        total_loss += unregularized_loss.item()
        if batch % LOG_INTERVAL == 0 and batch > 0:
            cur_loss = total_loss / LOG_INTERVAL
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:3.2E} | ms/batch {:5.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args.bptt_seq_len, lr,
                elapsed * 1000 / LOG_INTERVAL, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [17]:
best_val_loss = 1e20
try:
    for epoch in range(1, args.epochs + 1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(valid_data, args.eval_batch_size)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
            epoch, (time.time() - epoch_start_time), val_loss, math.exp(val_loss)))
        print('-' * 89)
        if val_loss < best_val_loss:
            with open(args.save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            lr *= LR_ANNEALING_RATE
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

/usr/local/lib/python3.6/site-packages/torch/nn/modules/module.py:477: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)


| epoch   1 |   200/  745 batches | lr 1.00E-03 | ms/batch 470.48 | loss  7.31 | ppl  1491.36
| epoch   1 |   400/  745 batches | lr 1.00E-03 | ms/batch 461.35 | loss  6.67 | ppl   791.08
| epoch   1 |   600/  745 batches | lr 1.00E-03 | ms/batch 456.92 | loss  6.44 | ppl   627.83
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 391.69s | valid loss  5.90 | valid ppl   364.23
-----------------------------------------------------------------------------------------
| epoch   2 |   200/  745 batches | lr 1.00E-03 | ms/batch 470.31 | loss  6.16 | ppl   473.85
| epoch   2 |   400/  745 batches | lr 1.00E-03 | ms/batch 450.27 | loss  6.04 | ppl   418.23
| epoch   2 |   600/  745 batches | lr 1.00E-03 | ms/batch 458.43 | loss  5.91 | ppl   369.47
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 392.57s | valid loss  5.50 | valid ppl   245.66
----------------

In [18]:
# Load the best saved model.
with open(args.save, 'rb') as f:
    model = torch.load(f, map_location=device)

In [20]:
# Run on test data.
test_loss = evaluate(test_data, args.test_batch_size)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  4.29 | test ppl    72.90
